<a href="https://colab.research.google.com/github/johnJay1745/CeDeFi/blob/main/SSETH_%26_SSLONG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
import numpy as np
import scipy.stats as st
import json
import requests
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from datetime import datetime

In [ ]:
pool_ids = {
    "Uni_V3_rETH-ETH": "5685fa0d-a7cf-4b60-9b77-ff23b4583f81",
    "Convex_FRAX-USDC":"844a8a29-4653-42e6-b675-a6b43fac3678",
    "ETH_AAVE": "f2726d05-1f8d-4b9c-80e3-43d03d85d117",
    "USDC_AAVE": "a349fea4-d780-4e16-973e-70ca9b606db2"
}

In [ ]:
def get_hist_apy(pool_id):
    data = requests.get("https://yields.llama.fi/chart/" + pool_id)
    data_json = data.json() 
    l = []
    for apy in data_json["data"]:
        l.append([apy["timestamp"],apy["apy"]])
    return pd.DataFrame(l).rename(columns = {0:"timestamp",1:"annual_ROI"})

In [ ]:
def get_coin_price(coin,vs_currency,days):
    data = requests.get("https://api.coingecko.com/api/v3/coins/" + 
                        coin +
                        "/market_chart?vs_currency=" +
                        vs_currency +
                        "&days=" + 
                        days)
    price_list = list(np.array(data.json()["prices"])[:,1])
    return pd.DataFrame(price_list)

In [ ]:
def get_leverage_ratio(coin,vs_currency,days):
    leverage_ratios = []
    df = pd.DataFrame(get_coin_price(coin,vs_currency,days))
    for i in range(len(df)):
        ETH_ratio = 0.25
        ETH_ratio = ETH_ratio * (df[0][i] / df[0][0])
        leverage_ratio = ETH_ratio / 1
        leverage_ratios.append(leverage_ratio)
    return pd.DataFrame(leverage_ratios)

In [ ]:
def get_HF_SLETH(coin,vs_currency,days,HF):
    HF_list = []
    df = 1 / pd.DataFrame(get_coin_price(coin,vs_currency,days))
    for i in range(len(df)):
       current_HF = HF
       current_HF = current_HF * (df[0][0] / df[0][i])
       HF_list.append(current_HF)
    return HF_list

In [ ]:
def SSETH_yield_df(HF,coin,vs_currency,days):
    df = pd.DataFrame()
    df["Time"] = get_hist_apy(pool_ids["Uni_V3_rETH-ETH"])["timestamp"]
    df["UNI_V3_returns"] = (get_hist_apy(pool_ids["Uni_V3_rETH-ETH"])
                           ["annual_ROI"])
    df["FRAX-USDC_returns"] = (get_hist_apy(pool_ids["Convex_FRAX-USDC"])
                               ["annual_ROI"])
    df["USDC_AAVE_returns"] = get_hist_apy(pool_ids["USDC_AAVE"])["annual_ROI"]
    df["ETH_AAVE_cost"] =  (- get_hist_apy(pool_ids["ETH_AAVE"])
                               ["annual_ROI"] / 
                               0.9)
    return df

In [ ]:
def SSETH_daily_yield(HF,coin,vs_currency,days):
    df = SSETH_yield_df(HF,coin,vs_currency,days)
    df["Daily_yield"] = (df["UNI_V3_returns"] * 
                         (get_leverage_ratio(coin,vs_currency,days)[0]) +
                         df["USDC_AAVE_returns"] + 
                         df["ETH_AAVE_cost"] * 
                         (2 * get_leverage_ratio(coin,vs_currency,days)[0]) + 
                         df["FRAX-USDC_returns"] * 
                         (0.5 / HF)
                        ) / 365
    return df

In [ ]:
def SSETH_price_move_ROI(coin,vs_currency,days,HF):
    df = pd.DataFrame()
    df[coin + "_price"] = get_coin_price(coin,vs_currency,days)
    df["Price_change"] = ((1 / df[coin + "_price"].shift(-1)) / 
                         (1 / df[coin + "_price"])) - 1
    df["PnL_ROI"] = df["Price_change"] * (0.5 / HF)
    return df

In [ ]:
def SSETH_total_return(coin,vs_currency,days,HF,pool_ids):
    l = []
    strategy_return = (SSETH_price_move_ROI(coin,vs_currency,days,HF)
                       ["PnL_ROI"] + 
                       SSETH_daily_yield(HF,coin,vs_currency,days)
                       ["Daily_yield"])
    for i in range(len(strategy_return)): 
        init_value = 1
        init_value = init_value * (1 + strategy_return[i])
        l.append(init_value)
                                   
    return pd.DataFrame(l)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x = SSETH_daily_yield(2,"ethereum","USD","229")["Time"], 
               y = SSETH_total_return("ethereum","USD","229",2,pool_ids)[0],
               name = "Position value (Short)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x = SSETH_daily_yield(2,"ethereum","USD","229")["Time"], 
               y = get_coin_price("ethereum","USD","229")[0],
               name = "ETH price"),
    secondary_y=True,
)
fig.update_layout(
    title = "SSETH return vs ETH price",
    yaxis_range = [0.85,1.15]
    # autosize = False,
    # width = 1000,
    # height = 500
)
fig.update_yaxes(title_text="ETH price", secondary_y = True)
fig.update_yaxes(title_text="Returns", secondary_y = False)

In [ ]:
def SLETH_yield_df(HF):
    df = pd.DataFrame()
    df["Time"] = get_hist_apy(pool_ids["Uni_V3_rETH-ETH"])["timestamp"]
    df["UNI_V3_returns"] = (get_hist_apy(pool_ids["Uni_V3_rETH-ETH"])
                           ["annual_ROI"]) 
    df["ETH_AAVE_returns"] = get_hist_apy(pool_ids["ETH_AAVE"])["annual_ROI"]
    df["USDC_borrow_cost"] =  (- get_hist_apy(pool_ids["USDC_AAVE"])
                               ["annual_ROI"] / 
                               0.9) 
    return df

In [ ]:
def SLETH_daily_yield(HF):
    df = SLETH_yield_df(HF)
    return (df["UNI_V3_returns"] * 
            (1 / HF) +
            df["ETH_AAVE_returns"] + 
            df["USDC_borrow_cost"] * 
            (1 / HF)
           ) / 365

In [ ]:
def SLETH_price_move_ROI(coin,vs_currency,days,HF):
    df = pd.DataFrame()
    df[coin + "_price"] = get_coin_price(coin,vs_currency,days)
    df["PnL_ROI"] = ((df[coin + "_price"].shift(-1) / df[coin + "_price"]) - 
                     1) * (1 / HF)
    return df

In [ ]:
def SLETH_total_return(coin,vs_currency,days,HF,pool_ids):
    l = []
    strategy_return = (SLETH_price_move_ROI(coin,vs_currency,days,HF)
                       ["PnL_ROI"] + 
                       SLETH_daily_yield(HF))

    for i in range(len(strategy_return)): 
        init_value = 1
        init_value = init_value * (1 + strategy_return[i])
        l.append(init_value)
           
    return [l,strategy_return]

In [ ]:
def rebalance(HF,price_impact,coin,vs_currency,days):
    df = pd.DataFrame()
    df["strategy_return"] = (SLETH_total_return
                             (coin,vs_currency,days,HF,pool_ids)
                             [0])
    df["HF"] = get_HF_SLETH(coin,vs_currency,days,HF)
    for i in range(len(df)):
        optimal_HF = HF
        if df["HF"][i] <= optimal_HF * (1.5 / 2):
             df["strategy_return"][i] = (df["strategy_return"][i] * 
                                         (1 + price_impact) * 
                                         (1 - df["strategy_return"][i] 
                                          / optimal_HF))
             optimal_HF = df["strategy_return"][i]
        elif df["HF"][i] >= optimal_HF * (2.15 / 2):
             df["strategy_return"][i] = (df["strategy_return"][i] * 
                                         (1 + price_impact) * 
                                         (df["strategy_return"][i] 
                                          / optimal_HF  
                                          - 1))
             optimal_HF = df["strategy_return"][i]
    return df

In [ ]:
rebalance(2,-0.003,"ethereum","USD","229")

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x = SLETH_yield_df(2)["Time"], 
               y = SLETH_total_return("ethereum",
                                   "USD",
                                   f"{len(SLETH_yield_df(2))}",
                                   2,pool_ids)[0],
               name = "Position value (Long)"),
    secondary_y = False
)

fig.add_trace(
    go.Scatter(x = SLETH_yield_df(2)["Time"], 
               y = SLETH_price_move_ROI("ethereum",
                                     "USD",
                                     f"{len(SLETH_yield_df(2))}",
                                     2)["ethereum_price"],
               name = "ETH price"),
    secondary_y = True
)

fig.update_layout(
    title = "SLETH return vs ETH price"
    # xaxis_title = "Time"
    # autosize = False,
    # width = 1000,
    # height = 500
)
fig.update_yaxes(title_text="ETH price", secondary_y = True)
fig.update_yaxes(title_text="Return", secondary_y = False)

In [ ]:
def get_data_frame():
    df = pd.DataFrame()
    # df["Time"] = SLETH_yield_df(2)["Time"]
    df["SLETH_retun"] = SLETH_total_return(
                        "ethereum",
                        "USD",
                        f"{len(SLETH_yield_df(2))}",
                        2,pool_ids)[0]
    df["SSETH_return"] = SSETH_total_return(
                         "ethereum",
                         "USD",
                         "229",
                         2,
                         pool_ids)[0]
    return df.to_csv("SSETH/SLETH_returns_csv.csv")

In [ ]:
get_data_frame()